In [ ]:
# For Microsoft SQL
# 1. In VSCode install the SQL Server Extension 
# 2. Connect to the Database using provided credentials (UoE wifi or using a VPN)
# 3. Run the first three commands below
# 4. Run the remaining ones in Python 

In [ ]:
# Table creation (adheres to 3NF)
-- 1. Department Table
CREATE TABLE Department (
    DepartmentID INT PRIMARY KEY IDENTITY(1,1),
    DepartmentName NVARCHAR(100) NOT NULL,
    BudgetAllocated DECIMAL(10,2),
    BudgetUsed DECIMAL(10,2)
);

-- 2. Employee Table
CREATE TABLE Employee (
    EmployeeID INT PRIMARY KEY IDENTITY(1,1),
    Age INT,
    Gender NVARCHAR(10),
    Attrition BIT,                 -- BIT (0/1) for yes/no
    BusinessTravel NVARCHAR(50),   -- e.g. 'Travel_Rarely', 'Non_Travel'
    JobLevel INT,
    EmploymentType NVARCHAR(50),   -- e.g. 'FullTime', 'PartTime', 'Intern'
    DepartmentID INT,
    FOREIGN KEY (DepartmentID) REFERENCES Department(DepartmentID)
);

-- 3. JobRole Table
CREATE TABLE JobRole (
    JobRoleID INT PRIMARY KEY IDENTITY(1,1),
    JobRoleName NVARCHAR(100) NOT NULL
);

-- 4. EmployeeJobRole (Link) Table - Now Includes DepartmentID
CREATE TABLE EmployeeJobRole (
    EmployeeID INT,
    JobRoleID INT,
    DepartmentID INT,
    PRIMARY KEY (EmployeeID, JobRoleID, DepartmentID),
    FOREIGN KEY (EmployeeID) REFERENCES Employee(EmployeeID),
    FOREIGN KEY (JobRoleID) REFERENCES JobRole(JobRoleID),
    FOREIGN KEY (DepartmentID) REFERENCES Department(DepartmentID)
);

-- 5. Compensation Table
CREATE TABLE Compensation (
    CompensationID INT PRIMARY KEY IDENTITY(1,1),
    EmployeeID INT UNIQUE,
    MonthlyIncome DECIMAL(10,2),
    OverTime BIT,                 -- BIT (0/1) for yes/no
    TravelAllowancePerYear DECIMAL(10,2),
    PercentSalaryHike INT,
    FOREIGN KEY (EmployeeID) REFERENCES Employee(EmployeeID)
);

-- 6. Performance Table
CREATE TABLE Performance (
    PerformanceID INT PRIMARY KEY IDENTITY(1,1),
    EmployeeID INT UNIQUE,
    JobSatisfaction INT,          -- rating scale (1–5)
    PerformanceRating INT,
    TrainingTimesLastYear INT,
    YearsAtCompany INT,
    FOREIGN KEY (EmployeeID) REFERENCES Employee(EmployeeID)
);

In [ ]:
# Populate the Department Table
SET IDENTITY_INSERT Department ON;  -- Enable manual ID insert

INSERT INTO Department (DepartmentID, DepartmentName, BudgetAllocated, BudgetUsed)
VALUES 
    (1, 'Sales', 200000, 179196),
    (2, 'Human Resources', 200000, 160701),
    (3, 'Research & Development', 200000, 192353);

SET IDENTITY_INSERT Department OFF; -- Restore normal auto-increment behavior


In [ ]:
# Populate the Job Role Table
INSERT INTO JobRole (JobRoleName)
VALUES 
    ('HRA'),
    ('Manager'),
    ('Sales Executive'),
    ('Junior Sales Associate'),
    ('Research Scientist'),
    ('Laboratory Technician'),
    ('Manufacturing Director'),
    ('Research Director'),
    ('Healthcare Representative');


In [12]:
# Populate the Employee Table
import pyodbc
import pandas as pd

# Read data from Excel file
df = pd.read_excel('dataset.xlsx', sheet_name='dataset')

# Connect to SQL Server
conn = pyodbc.connect(
    "DRIVER={SQL Server};" 
    "SERVER=mcruebs04.isad.isadroot.ex.ac.uk;" 
    "DATABASE=BEMM459_GroupZ;" 
    "UID=GroupZ;" 
    "PWD=YigS434*Wn"
)
cursor = conn.cursor()

# Fetch Department IDs dynamically
cursor.execute("SELECT DepartmentID, DepartmentName FROM Department")
dept_mapping = {row.DepartmentName: row.DepartmentID for row in cursor.fetchall()}

# Prepare the employee data
df_employee = df[['EmployeeID', 'Age', 'Gender', 'JobLevel', 'EmploymentType', 'Attrition', 'BusinessTravel', 'Department']].copy()

# Convert Attrition from "Yes"/"No" to 1/0
df_employee['Attrition'] = df_employee['Attrition'].map({'Yes': 1, 'No': 0})

# Map the Department column to DepartmentID
df_employee['DepartmentID'] = df_employee['Department'].map(dept_mapping)

# Prepare the DataFrame for insertion
df_employee_to_insert = df_employee[['EmployeeID', 'Age', 'Gender', 'JobLevel', 'EmploymentType', 'Attrition', 'BusinessTravel', 'DepartmentID']]

try:
    # Enable identity insertion 
    cursor.execute("SET IDENTITY_INSERT Employee ON")
    
    # Prepare the insert statement
    insert_sql = """
    INSERT INTO Employee (
        EmployeeID, Age, Gender, JobLevel, EmploymentType, 
        Attrition, BusinessTravel, DepartmentID
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """
    
    # Convert the DataFrame rows into a list of tuples for insertion
    employee_data = list(df_employee_to_insert.itertuples(index=False, name=None))
    
    # Insert the data into the Employee table
    cursor.executemany(insert_sql, employee_data)
    
    # Commit the transaction
    conn.commit()
    
    # Disable identity insertion
    cursor.execute("SET IDENTITY_INSERT Employee OFF")
    
    print("Employee table populated successfully!")
    
    # Verify the number of rows inserted
    cursor.execute("SELECT COUNT(*) AS EmployeeCount FROM Employee")
    print(f"Total Employees Inserted: {cursor.fetchone().EmployeeCount}")

except Exception as e:
    print("Error:", e)
    # Rollback in case of error
    conn.rollback()

finally:
    # Close cursor and connection
    cursor.close()
    conn.close()

All records deleted from Employee table.
Employee table repopulated successfully!

Department Mapping:
Sales: 1
Human Resources: 2
Research & Development: 3

Verification:
Total Employees Inserted: 68


In [16]:
# Populate the Compensation Table
# Select only the columns needed for the Compensation table
df_comp = df[['EmployeeID', 'MonthlyIncome', 
              'TravelAllowancePerYear', 'OverTime', 
              'PercentSalaryHike']].copy()

# Convert OverTime "Yes"/"No" to 1/0 for BIT type in SQL Server
df_comp['OverTime'] = df_comp['OverTime'].map({'Yes': 1, 'No': 0})

# Connect to SQL Server
conn = pyodbc.connect(
    "DRIVER={SQL Server};"
    "SERVER=mcruebs04.isad.isadroot.ex.ac.uk;"
    "DATABASE=BEMM459_GroupZ;"
    "UID=GroupZ;"
    "PWD=YigS434*Wn"
)
cursor = conn.cursor()

try:
    # Insert data into Compensation
    insert_sql = """
    INSERT INTO Compensation 
    (EmployeeID, MonthlyIncome, TravelAllowancePerYear, OverTime, 
     PercentSalaryHike)
    VALUES (?, ?, ?, ?, ?)
    """
    
    # Convert rows to tuples for executemany
    comp_data = list(df_comp.itertuples(index=False, name=None))
    
    cursor.executemany(insert_sql, comp_data)
    conn.commit()
    
    print("Compensation table populated successfully!")
    
    # Optional: Verify number of rows inserted
    cursor.execute("SELECT COUNT(*) AS CompensationCount FROM Compensation")
    print(f"Total Compensation Records Inserted: {cursor.fetchone().CompensationCount}")

except Exception as e:
    print("Error:", e)
    conn.rollback()

finally:
    cursor.close()
    conn.close()

Compensation table populated successfully!
Total Compensation Records Inserted: 68


In [17]:
# Populate Performance Table
# Select only the necessary columns for the Performance table
df_perf = df[['EmployeeID', 'JobSatisfaction', 'PerformanceRating', 
              'TrainingTimesLastYear', 'YearsAtCompany']].copy()

# Connect to SQL Server
conn = pyodbc.connect(
    "DRIVER={SQL Server};"
    "SERVER=mcruebs04.isad.isadroot.ex.ac.uk;"
    "DATABASE=BEMM459_GroupZ;"
    "UID=GroupZ;"
    "PWD=YigS434*Wn"
)
cursor = conn.cursor()

try:
    # Insert data into Performance table
    insert_sql = """
    INSERT INTO Performance 
    (EmployeeID, JobSatisfaction, PerformanceRating, TrainingTimesLastYear, YearsAtCompany)
    VALUES (?, ?, ?, ?, ?)
    """

    # Convert rows to tuples for executemany
    perf_data = list(df_perf.itertuples(index=False, name=None))

    cursor.executemany(insert_sql, perf_data)
    conn.commit()

    print("✅ Performance table populated successfully!")

except Exception as e:
    print("❌ Error:", e)

finally:
    cursor.close()
    conn.close()


✅ Performance table populated successfully!


In [18]:
# Populate Link Table
# Connect to SQL Server
conn = pyodbc.connect(
    "DRIVER={SQL Server};"
    "SERVER=mcruebs04.isad.isadroot.ex.ac.uk;"
    "DATABASE=BEMM459_GroupZ;"
    "UID=GroupZ;"
    "PWD=YigS434*Wn"
)
cursor = conn.cursor()

try:
    # Fetch JobRole mapping
    cursor.execute("SELECT JobRoleID, JobRoleName FROM JobRole")
    job_role_dict = {row.JobRoleName: row.JobRoleID for row in cursor.fetchall()}

    # Fetch Department mapping
    cursor.execute("SELECT DepartmentID, DepartmentName FROM Department")
    dept_dict = {row.DepartmentName: row.DepartmentID for row in cursor.fetchall()}

    # Prepare data for EmployeeJobRole 
    # Include Department from the dataset
    df_link = df[['EmployeeID', 'JobRole', 'Department']]
    
    # Create link data with EmployeeID, JobRoleID, and DepartmentID
    link_data = [
        (row['EmployeeID'], job_role_dict[row['JobRole']], dept_dict[row['Department']])
        for _, row in df_link.iterrows() 
        if row['JobRole'] in job_role_dict and row['Department'] in dept_dict
    ]

    # Insert into EmployeeJobRole with composite key
    insert_sql = """
    INSERT INTO EmployeeJobRole (EmployeeID, JobRoleID, DepartmentID) 
    VALUES (?, ?, ?)
    """
    
    cursor.executemany(insert_sql, link_data)
    conn.commit()
    
    print("✅ EmployeeJobRole table populated successfully!")
    
    # Verify insertion
    cursor.execute("SELECT COUNT(*) AS LinkCount FROM EmployeeJobRole")
    print(f"Total Links Inserted: {cursor.fetchone().LinkCount}")

except Exception as e:
    print("❌ Error:", e)
    conn.rollback()

finally:
    cursor.close()
    conn.close()

✅ EmployeeJobRole table populated successfully!
Total Links Inserted: 68


In [1]:
# For MongoDB
# 1. Ensure MongoDB Community Server 8.0.6 is installed
# 2. Start MongoDB in Command Prompt using "net start MongoDB"
# 3. Execute the code below

# Compatibility Requirements:
# pymongo: 4.11.3
# pandas: 2.2.2
from pymongo import MongoClient
import pandas as pd

In [3]:
# Establish MongoDB connection
client = MongoClient("mongodb://localhost:27017/")
db = client["BudgetCompensation"]

# Collections for employee feedback and manager feedback
employee_collection = db["EmployeeFeedback"]
manager_collection = db["ManagerFeedback"]

# Read the data from your Excel file (sheet "dataset")
df = pd.read_excel("dataset.xlsx", sheet_name="dataset")

# Prepare DataFrames for employee and manager feedback
employee_feedback_df = df[[
    "EmployeeID", "Gender", "EmployeeFeedback", "Attrition",
    "Department", "JobRole", "JobSatisfaction", "MonthlyIncome", "YearsAtCompany"
]]
manager_feedback_df = df[[
    "EmployeeID", "ManagerFeedback", "Department", "JobRole",
    "PerformanceRating", "PercentSalaryHike", "MonthlyIncome", "YearsAtCompany"
]]

# Convert DataFrames to lists of dictionaries
employee_docs = employee_feedback_df.to_dict(orient="records")
manager_docs = manager_feedback_df.to_dict(orient="records")

# Insert documents into collections
employee_collection.insert_many(employee_docs)
print("✅ Inserted into EmployeeFeedback collection:", employee_collection.count_documents({}))
manager_collection.insert_many(manager_docs)
print("✅ Inserted into ManagerFeedback collection:", manager_collection.count_documents({}))


✅ Inserted into EmployeeFeedback collection: 68
✅ Inserted into ManagerFeedback collection: 68
